### Project 6 - Exceptions

Suppose we have a Widget online sales application and we are writing the backend for it. We want a base `WidgetException` class that we will use as the base class for all our custom exceptions we raise from our Widget application.

Furthermore we have determined that we will need the following categories of exceptions:

```
1. Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
2. Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

Write an exception class hierarchy to capture this. In addition, we would like to implement the following functionality:
* implement separate internal error message and user error message
* implement an http status code associated to each exception type (keep it simple, use a 500 (server error) error for everything except invalid coupon code, and cannot stack coupons, these can be 400 (bad request)
* implement a logging function that can be called to log the exception details, time it occurred, etc.
* implement a function that can be called to produce a json string containing the exception details you want to display to your user (include http status code (e.g. 400), the user error message, etc)

##### Bonus

Log the traceback too - you'll have to do a bit of research for that! 

I'm going to use the `TracebackException` class in the `traceback` module:

https://docs.python.org/3/library/traceback.html#tracebackexception-objects

In particular, look at the class method `from_exception` (and remember that inside your exception class, the exception will be `self`!) and the `format` instance method. That method returns a generator, so you'll need to `list` it to print out everything in that traceback.

Good luck!

In [33]:
import json
from datetime import datetime
from http import HTTPStatus
import traceback

import executing.executing

In [34]:
class WidgetException(Exception):
    msg = "An error occurred in the Widget application."
    http_status_code = HTTPStatus.INTERNAL_SERVER_ERROR

    def __init__(self, *args, custom_msg=None):
        super().__init__(args)

        if args:
            self.msg =args[0]
        self.custom_msg = custom_msg if custom_msg else self.msg

    @property
    def traceback(self):\
        return traceback.TracebackException.from_exception(self).format()

    def log(self):
        excprtion = {
            "type": type(self).__name__,
            "message": self.msg,
            "arguments": self.args,
        }
        print(f"Logging Exception:{datetime.now().isoformat()}: {excprtion}")

    def to_json(self):
        response = {
            "http_status_code": self.http_status_code,
            "user_message": self.custom_msg,
            "timestamp": datetime.now().isoformat(),
        }
        return json.dumps(response)

In [35]:
ex1 =WidgetException("An example of a widget exception",10)

In [36]:
ex1.log()

Logging Exception:2025-03-29T21:00:37.072581: {'type': 'WidgetException', 'message': 'An example of a widget exception', 'arguments': (('An example of a widget exception', 10),)}


In [37]:
class SupplierException(WidgetException):
    msg = "A supplier error occurred."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)

class NotManufactoredException(SupplierException):
    msg = "The widget is not manufactured anymore."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)

class ProductionDelayedException(SupplierException):
    msg = "The widget production is delayed."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)

class ShippingDelayedException(SupplierException):
    msg = "The widget shipping is delayed."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)


class CheckoutException(WidgetException):
    msg = "A checkout error occurred."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)

class InventoryException(CheckoutException):
    msg = "An inventory error occurred."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)

class OutOfStockException(InventoryException):
    msg = "The widget is out of stock."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)


class PricingException(CheckoutException):
    msg = "A pricing error occurred."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)

class InvalidCouponException(PricingException):
    msg = "The coupon code is invalid."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)

class CannotStackCouponsException(PricingException):
    msg = "Coupons cannot be stacked."

    def __init__(self, *args, custom_msg=None):
        super().__init__(*args, custom_msg=custom_msg)

In [38]:
try:
    raise CannotStackCouponsException("Cannot stack coupons error")
except CannotStackCouponsException as e:
    e.log()
    print(e.to_json())
    raise

Logging Exception:2025-03-29T21:00:37.109159: {'type': 'CannotStackCouponsException', 'message': 'Cannot stack coupons error', 'arguments': (('Cannot stack coupons error',),)}
{"http_status_code": 500, "user_message": "Cannot stack coupons error", "timestamp": "2025-03-29T21:00:37.109159"}


CannotStackCouponsException: ('Cannot stack coupons error',)

In [32]:
try:
    raise ValueError
except ValueError as e:
    try:
        raise WidgetException("An example of a widget exception")
    except WidgetException as we:
        trace= traceback.TracebackException.from_exception(we)
        print(''.join(trace.format()))

Traceback (most recent call last):
  File "C:\Users\wb560607\AppData\Local\Temp\ipykernel_11784\1800236700.py", line 2, in <module>
    raise ValueError
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\wb560607\AppData\Local\Temp\ipykernel_11784\1800236700.py", line 5, in <module>
    raise WidgetException("An example of a widget exception")
WidgetException: ('An example of a widget exception',)



In [40]:
try:
    raise ValueError
except ValueError as e:
    try:
        raise WidgetException("An example of a widget exception")
    except WidgetException as we:
        trace= traceback.TracebackException.from_exception(we)
        print(''.join(we.traceback))

Traceback (most recent call last):
  File "C:\Users\wb560607\AppData\Local\Temp\ipykernel_11784\4007854433.py", line 2, in <module>
    raise ValueError
ValueError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\wb560607\AppData\Local\Temp\ipykernel_11784\4007854433.py", line 5, in <module>
    raise WidgetException("An example of a widget exception")
WidgetException: ('An example of a widget exception',)

